In [2]:
# Import necessary modules
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# Load environment variables from .env file
# Ensure the .env file is in the parent directory or specify the path if elsewhere
dotenv_path = os.path.join(os.path.dirname('__file__'), '..', '.env') # Assumes .env is one level up
load_dotenv(dotenv_path=dotenv_path)
print(f"Attempting to load .env from: {dotenv_path}") # Debugging print
print(f".env loaded: {load_dotenv(dotenv_path=dotenv_path)}") # Check if loading was successful

# Establish database connection using credentials from environment variables
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

# Check if all necessary variables are loaded
if None in (DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME):
    # Print which variables are missing for debugging
    missing_vars = [var for var, val in {'DB_USER': DB_USER, 'DB_PASSWORD': DB_PASSWORD, 'DB_HOST': DB_HOST, 'DB_PORT': DB_PORT, 'DB_NAME': DB_NAME}.items() if val is None]
    raise Exception(f"Database credentials not fully set. Missing: {', '.join(missing_vars)}")

print("Database credentials loaded successfully.") # Confirmation

try:
    # Create SQLAlchemy engine
    conn_string = f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
    engine = create_engine(conn_string)
    # Test connection
    with engine.connect() as connection:
        print("Database connection successful!")
except Exception as e:
    print(f"Database connection failed: {e}")
    raise

# Set pandas display options (optional, you already have this later)
pd.set_option('display.max_rows', None)

Attempting to load .env from: ..\.env
.env loaded: True
Database credentials loaded successfully.
Database connection successful!


# Descriptive Analytics Query (API Data)

### Business Question:
Which GICS sector in the S&P 500 has experienced the largest year-over-year increase in daily price volatility since 2014 and when did it occur?

In [3]:
# Descriptive Analytics Query for API Data
sql_query = '''
WITH daily_returns AS (
    -- Keep existing daily returns calculation
    SELECT 
        s.symbol,
        s.gics_sector,
        d.year,
        d.trade_date,
        fp.close_price,
        LAG(fp.close_price, 1) OVER (PARTITION BY s.symbol ORDER BY d.trade_date) AS prev_close_price,
        (fp.close_price - LAG(fp.close_price, 1) OVER (PARTITION BY s.symbol ORDER BY d.trade_date)) / 
        LAG(fp.close_price, 1) OVER (PARTITION BY s.symbol ORDER BY d.trade_date) AS daily_return
    FROM fact_price fp
    JOIN dim_symbol s ON fp.symbol_id = s.symbol_id
    JOIN dim_date d ON fp.date_id = d.date_id
    WHERE d.year >= 2014
),
annual_volatility AS (
    -- Keep existing annual volatility calculation
    SELECT 
        gics_sector,
        year,
        COUNT(DISTINCT symbol) as num_stocks,
        COUNT(DISTINCT trade_date) as trading_days,
        STDDEV(daily_return) * SQRT(252) as annual_volatility
    FROM daily_returns
    WHERE daily_return IS NOT NULL
    GROUP BY gics_sector, year
),
yoy_changes AS (
    -- Keep existing YoY calculations
    SELECT 
        av.gics_sector,
        av.year,
        av.num_stocks,
        av.trading_days,
        av.annual_volatility as current_volatility,
        LAG(av.annual_volatility) OVER (PARTITION BY av.gics_sector ORDER BY av.year) as prev_year_volatility,
        (av.annual_volatility - LAG(av.annual_volatility) OVER (PARTITION BY av.gics_sector ORDER BY av.year)) 
            / LAG(av.annual_volatility) OVER (PARTITION BY av.gics_sector ORDER BY av.year) * 100 as volatility_change_pct,
        (av.annual_volatility - LAG(av.annual_volatility) OVER (PARTITION BY av.gics_sector ORDER BY av.year)) as volatility_change_abs
    FROM annual_volatility av
)
SELECT 
    gics_sector,
    year,
    num_stocks,
    ROUND(current_volatility, 4) as current_volatility,
    ROUND(prev_year_volatility, 4) as prev_year_volatility,
    ROUND(volatility_change_pct, 2) as volatility_change_pct,
    ROUND(volatility_change_abs, 4) as volatility_change_abs
FROM yoy_changes
WHERE volatility_change_pct IS NOT NULL
    AND trading_days > 200  -- Filter out partial years like 2025
ORDER BY volatility_change_pct DESC
LIMIT 10;
'''


In [4]:
api_results = pd.read_sql(sql_query, engine)
api_results


,gics_sector,year,num_stocks,current_volatility,prev_year_volatility,volatility_change_pct,volatility_change_abs
0,Real Estate,2020,24,0.5258,0.1800,192.17,0.3458
1,Financials,2020,61,0.5607,0.2188,156.30,0.3420
2,Consumer Discretionary,2020,41,0.6732,0.2657,153.35,0.4075
3,Energy,2020,16,0.8461,0.3386,149.91,0.5075
4,Industrials,2020,60,0.4983,0.2522,97.60,0.2461
5,Materials,2020,22,0.5343,0.2834,88.51,0.2509
6,Communication Services,2020,17,0.4984,0.2682,85.87,0.2303
7,Health Care,2020,45,0.4745,0.2894,64.00,0.1852
8,Consumer Staples,2020,29,0.3698,0.2277,62.43,0.1422
9,Information Technology,2020,55,0.5338,0.3306,61.44,0.2031


### Data Dictionary

1.  **`gics_sector`**:
    *   **Definition:** The Global Industry Classification Standard (GICS) sector name. This broadly categorizes companies based on their primary business activity.
    *   **Example:** 'Real Estate', 'Financials', 'Energy'.

2.  **`year`**:
    *   **Definition:** The calendar year for which the `current_volatility` metric was calculated. In this table, it's specifically 2020.
    *   **Example:** 2020.

3.  **`num_stocks`**:
    *   **Definition:** The number of distinct companies within that `gics_sector` that had sufficient data available in the database to be included in the volatility calculation for the specified `year` (2020).
    *   **Example:** 24 (for Real Estate in 2020).

4.  **`current_volatility`**:
    *   **Definition:** The calculated annualized volatility of the sector's daily returns for the specified `year` (2020). It's typically calculated as the standard deviation of the sector's average daily returns, multiplied by the square root of the number of trading days in a year (commonly √252). A higher value indicates greater price fluctuation or risk during that year.
    *   **Example:** 0.5258 (or 52.58%) for Real Estate in 2020.

5.  **`prev_year_volatility`**:
    *   **Definition:** The calculated annualized volatility of the sector's daily returns for the year *prior* to the specified `year`. In this case, it represents the volatility for 2019.
    *   **Example:** 0.1800 (or 18.00%) for Real Estate in 2019.

6.  **`volatility_change_pct`**:
    *   **Definition:** The percentage change in annualized volatility from the previous year (2019) to the current year (2020). Calculated as `((current_volatility - prev_year_volatility) / prev_year_volatility) * 100`. This highlights the relative magnitude of the volatility increase or decrease.
    *   **Example:** 192.17% for Real Estate, indicating its 2020 volatility was 192.17% higher than its 2019 volatility.

7.  **`volatility_change_abs`**:
    *   **Definition:** The absolute difference in annualized volatility between the current year (2020) and the previous year (2019). Calculated as `current_volatility - prev_year_volatility`. This shows the raw change in the volatility measure.
    *   **Example:** 0.3458 for Real Estate, meaning its annualized volatility measure increased by approximately 0.35 from 2019 to 2020.



### Analysis:
*   **Insight:**
*   **Recommendation:** 
*   **Prediction:** 

# Diagnostic Analytics Query (API Data)

### Business Question:
Which sub-industries or individual companies within the Utilities sector drove the large volatility increase observed in 2020?

In [7]:
sql_query = '''
WITH daily_returns AS (
    SELECT 
        s.symbol,
        s.security,
        s.gics_industry,
        d.year,
        d.trade_date,
        fp.close_price,
        LAG(fp.close_price, 1) OVER (PARTITION BY s.symbol ORDER BY d.trade_date) AS prev_close_price,
        (fp.close_price - LAG(fp.close_price, 1) OVER (PARTITION BY s.symbol ORDER BY d.trade_date)) / 
        LAG(fp.close_price, 1) OVER (PARTITION BY s.symbol ORDER BY d.trade_date) AS daily_return
    FROM fact_price fp
    JOIN dim_symbol s ON fp.symbol_id = s.symbol_id
    JOIN dim_date d ON fp.date_id = d.date_id
    WHERE (s.gics_sector = 'Utilities' OR s.symbol = 'SPY')  -- Adding SPY ETF as benchmark
    AND d.year = 2020
),
company_metrics AS (
    SELECT 
        symbol,
        security,
        COALESCE(gics_industry, 'S&P 500 Index') as gics_industry,  -- Label for benchmark
        COUNT(DISTINCT trade_date) as trading_days,
        STDDEV(daily_return) * SQRT(252) as annual_volatility,
        (EXP(SUM(LN(1 + NULLIF(daily_return, 0)))) - 1) * 100 as annual_return_pct
    FROM daily_returns
    WHERE daily_return IS NOT NULL
    GROUP BY symbol, security, gics_industry
)
SELECT 
    symbol,
    security,
    gics_industry,
    ROUND(annual_volatility, 4) as annual_volatility,
    ROUND(annual_return_pct, 2) as annual_return_pct,
    trading_days
FROM company_metrics
ORDER BY 
    CASE WHEN symbol = 'SPY' THEN 0 ELSE 1 END,  -- Show benchmark first
    annual_volatility DESC;
'''


In [8]:
# Execute diagnostic query for API data
api_diag_results = pd.read_sql(sql_query, engine)
api_diag_results

,symbol,security,gics_industry,annual_volatility,annual_return_pct,trading_days
0,PCG,PG&E Corporation,Multi-Utilities,0.7378,14.82,252
1,CNP,CenterPoint Energy,Multi-Utilities,0.5857,-16.99,252
2,AES,AES Corporation,Independent Power Producers & Energy Traders,0.5812,21.46,252
3,FE,FirstEnergy,Electric Utilities,0.5288,-32.72,252
4,EVRG,Evergy,Electric Utilities,0.4922,-9.34,252
5,EXC,Exelon,Electric Utilities,0.4889,-3.57,252
6,NRG,NRG Energy,Independent Power Producers & Energy Traders,0.4790,0.25,252
7,DTE,DTE Energy,Multi-Utilities,0.4753,-2.02,252
8,SRE,Sempra,Multi-Utilities,0.4694,-11.41,252
9,PPL,PPL Corporation,Electric Utilities,0.4682,-15.65,252


### Data Dictionary
1. **symbol**
   - The stock ticker symbol used on exchanges
   - Example: 'CNP' for CenterPoint Energy

2. **security**
   - The full company name
   - Example: 'CenterPoint Energy'

3. **gics_industry**
   - The Global Industry Classification Standard sub-industry classification
   - Categories shown:
     - Multi-Utilities (companies operating in multiple utility services)
     - Independent Power Producers & Energy Traders
     - Electric Utilities
     - Water Utilities
     - Gas Utilities

4. **annual_volatility**
   - Measures the degree of variation in daily stock returns over 2020
   - Calculated as: Standard deviation of daily returns × √252 (annualization factor)
   - Higher numbers indicate more price volatility
   - Example: 0.5857 (58.57%) for CNP means very high price swings

5. **annual_return_pct**
   - The total percentage return for 2020
   - Calculated using compound daily returns: (EXP(SUM(LN(1 + daily_return))) - 1) * 100
   - Positive numbers indicate gains, negative indicate losses
   - Example: -16.99% for CNP means investors lost 16.99% of value in 2020

6. **trading_days**
   - Number of days the stock traded during 2020
   - 252 is typical for a full year (excluding weekends and holidays)
   - Used to verify data completeness


### Analysis:
*   **Insight:**
*   **Recommendation:** 
*   **Prediction:** 